In [ ]:
#Rumus triangular fuzzy
def segitiga_naik(max:float,min:float,current:float) -> float:
    """
    Fungsi segitiga_naik digunakan untuk menghitung nilai fuzzy dari himpunan fuzzy segitiga naik
    
    Rumus : (current-min)/(max-min)
    current : nilai input
    min : nilai minimum dari himpunan fuzzy
    max : nilai maksimum dari himpunan fuzzy
    
    """
    return (current-min)/(max-min)

def segitiga_turun(max:float,min:float,current:float) -> float:
    """
    Fungsi segitiga_turun digunakan untuk menghitung nilai fuzzy dari himpunan fuzzy segitiga turun
    
    Rumus : (max-current)/(max-min)
    current : nilai input
    min : nilai minimum dari himpunan fuzzy
    max : nilai maksimum dari himpunan fuzzy
    """
    return (max-current)/(max-min)

In [ ]:
#Membuat domain fuzzy
def make_domain(min:float,max:float,amount:int) -> list:
    """
    Fungsi make_domain digunakan untuk membuat domain fuzzy
    
    min : nilai minimum dari domain fuzzy
    max : nilai maksimum dari domain fuzzy
    amount: jumlah domain fuzzy yang akan dibuat
    
    """#C
    
    domain = [(i, i-1+(max//amount)) for i in range(min,max+(max%amount),int(max/amount))]
    if len(domain) > amount or domain[-1][1] > max:
        domain.pop()
        domain[-1] = (domain[-1][0],max)
    
    if len(domain) != amount:
        raise ValueError("Kesalahan dalam pembuatan domain fuzzy. Nilai yang digunakan tidak dapat dibagi rata dengan jumlah domain yang ingin dibuat")
    
    return domain
    

In [24]:
#Pembuatan rules
def ceiling(x:float, y:float) -> int:
    """
    Fungsi ceiling digunakan untuk membulatkan nilai x ke atas
    
    x : nilai yang akan dibulatkan
    y : pembulatan
    
    """
    if(x % y == 0):
        return int(x//y)
    else:
        return int(x//y+1)
    
def flatten_list(list_flatten:list) -> list:
    """
    Fungsi flatten_list digunakan untuk meratakan list
    
    list_flatten : list yang akan diratakan
    
    """
    return [item for sublist in list_flatten for item in sublist]
    
def rules_recursion(list_rules:list) -> list:
    """
    Fungsi rules_recursion digunakan untuk membuat rules secara rekursif
    
    list_rules : list rules yang akan dibuat
    
    """
    if len(list_rules) == 1:
        return list_rules
    else:
        temp_rules = []
        for i in list_rules[0]:
            for j in list_rules[1]:
                temp_rules.append([i,j])
        list_rules.pop(0)
        list_rules.pop(0)
        # list_rules.insert(0,flatten_list(temp_rules))
        list_rules.insert(0,temp_rules)
        return rules_recursion(list_rules)

def split_list(list_to_split:list, n:int) -> list:
    """
    Fungsi split_list digunakan untuk membagi list menjadi n bagian
    
    list_to_split : list yang akan dibagi
    n : jumlah bagian
    
    """
    return [list_to_split[i:i + n] for i in range(0, len(list_to_split), n)]

def make_rules(domain:dict) -> list:
    """
    Fungsi make_rules digunakan untuk membuat rules dari domain fuzzy
    
    domain : domain dari tiap variabel fuzzy
    example : dict{'variabel1':3,'variabel2':3}
    
    """
    dict_tingkatfuzzy = {}
    banyak_domain = len(domain.items())
    
    for key,value in domain.items():
        dict_rules = {}
        if not isinstance(value,int):
            raise ValueError("Jumlah domain fuzzy harus berupa integer")
        
        #Ini buat nentuin rulesnya kalo dibuat secara dinamis
        tingkat_fuzzy = [i for i in range(1,value+1)]
        dict_tingkatfuzzy[key] = tingkat_fuzzy
    
    max_tingkatfuzzy = 0
    min_tingkatfuzzy = 0
    for key,value in dict_tingkatfuzzy.items():
        # print(value)
        max_tingkatfuzzy += max(value)
        min_tingkatfuzzy += min(value)
    
    # print(max_tingkatfuzzy, min_tingkatfuzzy)
        
    array_tingkatfuzzy = [i for i in range(min_tingkatfuzzy,max_tingkatfuzzy+1)]
    banyak_nilaitingkat = len(array_tingkatfuzzy)
    
    if(banyak_nilaitingkat % banyak_domain) != 0:
        # ini dipake kalo jumlah domain fuzzy ga bisa dibagi rata, misalnya 3 domain fuzzy
        
        dict_rules[ceiling(banyak_domain,2)] = [array_tingkatfuzzy[i] for i in range(ceiling(banyak_nilaitingkat,banyak_domain)-1, ceiling(banyak_nilaitingkat*2,banyak_domain))]
        for i in dict_rules[ceiling(banyak_domain,2)]:
            array_tingkatfuzzy.remove(i)
            
    #sisa tingkat fuzzy = banyak_domain - 1    
    
    for i in tingkat_fuzzy:
        if dict_rules.get(i) == None:
            added_rules = []
            for _ in range(0,banyak_domain-1):
                added_rules.append(array_tingkatfuzzy.pop(0))     
            dict_rules[i] = added_rules  
      
    print(dict_tingkatfuzzy)
    print(dict_rules)
    list_rules_nongroup = rules_recursion(list(dict_tingkatfuzzy.values()))
    list_rules_grouped = []
    #kombinasi dari semua tingkatfuzzy
    temp_rules = []
    
    for _ in range(banyak_domain-1):
        list_rules_nongroup=flatten_list(list_rules_nongroup)
        
    for key,value in enumerate(list_rules_nongroup):
        if type(value) == list:
            for j in value:
                temp_rules.append(j)
            temp_rules.append(list_rules_nongroup[key+1])
        else:
            list_rules_grouped.append(temp_rules)
            temp_rules=[]
            continue
        
    #Buat rules yang terakhir
    combined_rules = []
    for key,value in enumerate(list_rules_grouped):
        score = 0
        for i in value:
            score += i
        #cari index dari score pada dict rules
        for key2,value2 in dict_rules.items():
            if score in value2:
                combined_rules.append((value, key2))
        
            
        
    # #Making rules
    # rules = []
    return combined_rules
        
            
    

In [25]:
rules = make_rules({'variabel1':3,'variabel2':3,'variabel3':3})

{'variabel1': [1, 2, 3], 'variabel2': [1, 2, 3], 'variabel3': [1, 2, 3]}
{2: [5, 6, 7], 1: [3, 4], 3: [8, 9]}


In [23]:
# print([i for i in make_rules({'variabel1':3,'variabel2':2, 'variabel3':3})],sep='\n')
def map_rules(list_to_map:list=[],rules_map:dict={1:'Rendah', 2:'Sedang',3:'Tinggi'}) -> dict:
    """
    Fungsi map_rules digunakan untuk mengubah nilai rules dari integer ke string
    
    dict_rules : rules yang akan diubah
    example: dict_rules = {1: 'Rendah', 2: 'Sedang', 3: 'Tinggi'}
    
    
    """
    try: 
        return list(map(lambda x: (list(map(lambda y: rules_map[y], x[0])), rules_map[x[1]]), list_to_map))
    except Exception as e:
        print("Error: ",e)


# rules_map = {
#     1: 'Rendah',
#     2: 'Sedang',
#     3: 'Tinggi'
# }

rules = make_rules({'variabel1':3,'variabel2':3,'variabel3':3})
rules = map_rules(list_to_map=rules, rules_map={1:'S', 2:'M', 3:'H'})
# rules = list(map(lambda x: (list(map(lambda y: rules_map[y], x[0])), rules_map[x[1]]), rules))

print(len(rules))
for i in rules:
    print(i)
# print(ceiling(7*2, 3))
 
 
#hitung banyak M di rules
countm = 0
for i in rules:
    countm += (i[1].count('M'))
print(countm)

{'variabel1': [1, 2, 3], 'variabel2': [1, 2, 3], 'variabel3': [1, 2, 3]}
{2: [5, 6, 7], 1: [3, 4], 3: [8, 9]}
27
(['S', 'S', 'S'], 'S')
(['S', 'S', 'M'], 'S')
(['S', 'S', 'H'], 'M')
(['S', 'M', 'S'], 'S')
(['S', 'M', 'M'], 'M')
(['S', 'M', 'H'], 'M')
(['S', 'H', 'S'], 'M')
(['S', 'H', 'M'], 'M')
(['S', 'H', 'H'], 'M')
(['M', 'S', 'S'], 'S')
(['M', 'S', 'M'], 'M')
(['M', 'S', 'H'], 'M')
(['M', 'M', 'S'], 'M')
(['M', 'M', 'M'], 'M')
(['M', 'M', 'H'], 'M')
(['M', 'H', 'S'], 'M')
(['M', 'H', 'M'], 'M')
(['M', 'H', 'H'], 'H')
(['H', 'S', 'S'], 'M')
(['H', 'S', 'M'], 'M')
(['H', 'S', 'H'], 'M')
(['H', 'M', 'S'], 'M')
(['H', 'M', 'M'], 'M')
(['H', 'M', 'H'], 'H')
(['H', 'H', 'S'], 'M')
(['H', 'H', 'M'], 'H')
(['H', 'H', 'H'], 'H')
19


In [7]:
# User mau input domain, berapa ada kelas fuzzy
# ada 3 fitur, waktu belajar, lama jawab soal, nilai
# kesimpulan = lulus atau tidak
# lama jawab soal min 5 menit max 15 menit
# waktu belajar min 1 menit maksimal 60 menit
# nilai min 0 maks 100

#Waktu belajar
min_waktubelajar:int = 1
max_waktubelajar:int = 60
domain_waktubelajar:int = 3

#Lama jawab
min_lamajawab:int = 5
max_lamajawab:int = 15
domain_lamajawab:int = 3

#Nilai
min_nilai:int = 0
max_nilai:int = 100
domain_nilai:int = 3

# waktu_belajar = [(i, i+max_waktubelajar//domain_waktubelajar-1) for i in range(min_waktubelajar, max_waktubelajar+1, max_waktubelajar//domain_waktubelajar)]
# print(waktu_belajar)

# lama_jawab = [(i, i+max_lamajawab//domain_lamajawab-1) for i in range(1, max_lamajawab+1, max_lamajawab//domain_lamajawab)]
# print(lama_jawab)

# nilai = [(i, i+max_nilai//domain_nilai-1) for i in range(0, max_nilai+1, max_nilai//domain_nilai)]
# print(nilai)
print(make_domain(min_waktubelajar, max_waktubelajar, domain_waktubelajar))
print(make_domain(1, max_lamajawab, domain_lamajawab))
print(make_domain(min_nilai,max_nilai, domain_nilai))

[(1, 20), (21, 40), (41, 60)]
[(1, 5), (6, 10), (11, 15)]
[(0, 32), (33, 65), (66, 100)]
